# Preparation

In [4]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../..')

from trafficSimulator import *
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [5]:
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

vehicle_rate = 60
fast_track_factor, slow_track_factor = 1, 0.33
stop_distance = 25
fast_wait_time = 4
slow_wait_time = 4
vehicle_limit = 80

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [6]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [7]:
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

## For the connections of the topology, we can either represent with concrete lines
## or just as a dot (as shown below). Either case, the vehicles will not actually go 
## pass these routes; they are more for demonstration purposes

connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [8]:
# Smaller delta encourages more exploration
delta = 0.25

# Penalty proportion for delay caused
alpha = 2.5

# Interpolation factor between money and time
gamma_mean, gamma_var = 2.5, 0.25

# Level of risk aversion
eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 4s, sys: 4.97 s, total: 3min 9s
Wall time: 3min 3s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[2, 1]",36.4,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[0, 3]",101.95,"[-999, 13]","[0, 0]","[0, 0]"
2,2,"[2, 3]",68.5,"[0, -999]","[0, 0]","[0, 0]"
3,3,"[0, 1]",77.95,"[1, 22]","[3.049999999999997, 0]","[3.049999999999997, 0]"
4,4,"[0, 3]",112.9,"[3, 29]","[2.499999999999999, 0]","[2.499999999999999, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",142.5,"[74, 74]","[0, 0]","[0, 0]"
76,76,"[0, 1]",147.65,"[75, 77]","[2.4499999999999993, 0]","[2.4499999999999993, 0]"
77,77,"[2, 1]",152.55,"[73, 75]","[9.349999999999998, 0]","[0, 0]"
78,78,"[0, 1]",147.7,"[76, 76]","[0.7500000000000001, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[2, 1]",36.4,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[0, 3]",101.95,"[-999, 13]","[0, 0]","[0, 0]",1
2,2,"[2, 3]",68.5,"[0, -999]","[0, 0]","[0, 0]",1
3,3,"[0, 1]",77.95,"[1, 22]","[3.049999999999997, 0]","[3.049999999999997, 0]",1
4,4,"[0, 3]",112.9,"[3, 29]","[2.499999999999999, 0]","[2.499999999999999, 0]",1
5,5,"[2, 3]",71.2,"[2, 2]","[0, 0]","[0, 0]",1
6,6,"[2, 1]",41.1,"[5, 0]","[0, 0]","[0, 0]",1
7,7,"[2, 3]",70.95,"[6, 5]","[0, 0]","[0, 0]",1
8,8,"[2, 1]",41.6,"[7, 6]","[0, 0]","[0, 0]",1
9,9,"[2, 3]",71.0,"[8, 7]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Probabilities'])

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

In [14]:
etas = []
gammas = []

for i in range(len(utility_df)):
    eta = np.random.normal(eta_mean, eta_var, 1)[0]
    while eta < 0:
        eta = np.random.normal(eta_mean, eta_var, 1)[0]
    etas.append(eta)
    
for i in range(len(utility_df)):
    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    while gamma < 0:
        gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    gammas.append(gamma)

utility_df['Gamma'] = gammas
utility_df['Eta'] = etas

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],2.358908,1.110914
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],2.353538,1.222103
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],2.452734,0.721759
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],2.460881,1.280001
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],2.108153,0.915300


## Summary of the 1st round

In [15]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,3,0.00,1,36.40
1,1,0,3.05,1,101.95
2,2,2,0.00,1,68.50
3,3,1,2.50,1,77.95
4,4,0,0.00,1,112.90
5,5,2,0.00,1,71.20
6,6,3,0.00,1,41.10
7,7,2,0.00,1,70.95
8,8,3,0.00,1,41.60
9,9,2,0.00,1,71.00


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333333322450975, 0.33333333322450975, 0.3...",[3],2.358908,1.110914
1,1,"[-11.607741003118702, 16.651092223153956, 16.6...","[1.7791944291689836e-13, 0.333333333333274, 0....",[0],2.353538,1.222103
2,2,"[16.651092223153956, 16.651092223153956, -1.06...","[0.333333331085828, 0.333333331085828, 6.74251...",[2],2.452734,0.721759
3,3,"[16.651092223153956, -12.719811719097606, 16.6...","[0.33333333333331383, 5.851365606422622e-14, 0...",[1],2.460881,1.280001
4,4,"[-2.7141692489049207, 16.651092223153956, 16.6...","[1.296141889949888e-09, 0.33333333290128603, 0...",[0],2.108153,0.915300
5,5,"[16.651092223153956, 16.651092223153956, -2.44...","[0.3333333327666232, 0.3333333327666232, 1.700...",[2],2.406045,0.909049
6,6,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333327204315, 0.3333333327204315, 0.333...",[3],2.419640,0.941144
7,7,"[16.651092223153956, 16.651092223153956, -7.88...","[0.33333333333087545, 0.33333333333087545, 7.3...",[2],2.243687,1.227988
8,8,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333333292048645, 0.33333333292048645, 0.3...",[3],2.716069,0.971702
9,9,"[16.651092223153956, 16.651092223153956, -3.29...","[0.33333333309166024, 0.33333333309166024, 7.2...",[2],2.646306,0.979154


In [16]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [17]:
%%time

import time
time_l = []
begin = time.time()

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 100

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)
    
    time_l.append(round(time.time()-begin-sum(time_l), 2))

CPU times: user 6h 32min 39s, sys: 10min 27s, total: 6h 43min 6s
Wall time: 6h 28min 51s


In [18]:
utility_tmp.to_csv(f"Saved_data/Simulation_records/Summary.csv", index=False)